## Erstmal auf den Zyklus gruppieren und dann Features erstellen

### Import

In [1]:
import pandas as pd
import numpy as np

import os
import sys
import logging

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed\Zyklus'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

In [42]:
for file in os.listdir(p_fold):
    if ".csv" in file:
        logging.info(f"<<< Started processing file: {file} >>>")
        new_filename = file.replace("prepared","processed")
        
        df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                                 #nrows= 100000
                                )
        logging.info(f"<<< Size of Dataframe: {df.shape} >>>")
        
        #Auf Ebene Batterie + Zyklus gruppieren mit benötigten Aggregationen
        df_pv = pd.pivot_table(
           df,
           index=['batteryname','zyklus','comment','type'],
           aggfunc={'time': np.min, 'relativeTime': np.max, 'voltage': [np.min, np.max, np.mean], 'current': [np.min, np.max, np.mean], 'temperature': [np.min, np.max, np.mean]}
        ).reset_index()
        
        df_pv.columns = ['_'.join(col) for col in list(df_pv.columns)]
        
        
        logging.info(f"<<< Start saving dataframe with shape {df_pv.shape} >>>")
        df_pv.to_csv(f"{folder}\\{new_filename}", sep=',', index=False)
        logging.info(f"<<< Saved processed file in directory {folder}\\{new_filename} >>>")

[08:00:28] INFO: <<< Started processing file: prepared_RW1.csv >>>
[08:00:32] INFO: <<< Size of Dataframe: (2958840, 11) >>>
[08:00:33] INFO: <<< Start saving dataframe with shape (17065, 15) >>>
[08:00:33] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW1.csv >>>
[08:00:33] INFO: <<< Started processing file: prepared_RW10.csv >>>
[08:00:46] INFO: <<< Size of Dataframe: (8596025, 11) >>>
[08:00:49] INFO: <<< Start saving dataframe with shape (110818, 15) >>>
[08:00:50] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW10.csv >>>
[08:00:50] INFO: <<< Started processing file: prepared_RW11.csv >>>
[08:01:03] INFO: <<< Size of Dataframe: (8664510, 11) >>>
[08:01:05] INFO: <<< Start saving dataframe with shape (109389, 15) >>>
[08:01:06

In [43]:
df = pd.read_csv(f"{folder}\\{new_filename}")

In [44]:
df

,batteryname_,zyklus_,comment_,type_,current_amax,current_amin,current_mean,relativeTime_amax,temperature_amax,temperature_amin,temperature_mean,time_amin,voltage_amax,voltage_amin,voltage_mean
0,RW9,0,reference charge,C,-0.010,-2.007,-0.686090,10504.66,27.83544,21.54400,24.187009,5.04,4.200,3.838,4.168174
1,RW9,1,reference discharge,D,1.004,0.999,0.999910,7552.60,27.49199,20.49803,24.955130,10509.70,4.113,3.200,3.724370
2,RW9,2,reference charge,C,-0.010,-2.010,-0.718712,10513.58,27.74177,20.73221,24.046205,18062.29,4.200,3.492,4.160357
3,RW9,3,reference discharge,D,1.003,0.999,0.999903,7541.58,27.35149,21.73134,24.701164,28575.88,4.113,3.200,3.723958
4,RW9,4,pulsed load (rest),R,0.000,0.000,0.000000,1200.00,23.23005,22.91782,23.121925,46858.08,4.199,4.197,4.197756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113573,RW9,113573,rest post reference discharge,R,0.000,0.000,0.000000,7200.00,38.29132,30.73195,32.874813,12675446.09,3.802,3.328,3.778197
113574,RW9,113574,reference charge,C,-0.010,-2.000,-0.135870,20532.71,34.13290,30.76287,32.242548,12682646.12,4.200,4.096,4.199940
113575,RW9,113575,rest post reference charge,R,0.000,0.000,0.000000,300.00,32.47880,32.40151,32.446385,12703178.80,4.198,4.195,4.195839
113576,RW9,113576,reference discharge,D,1.005,0.999,0.999960,2700.84,38.86329,32.46334,35.958514,12703478.83,4.055,3.200,3.553618
